In [ ]:
import ipywidgets

# HTML refresher and GET requests

Before we can scrape HTML Pages, we need to learn little bit about the DOM (Document Object Model)

In [ ]:
url = "https://www.computerhope.com/jargon/d/dom1.jpg"
iframe = '<iframe src=' + url + ' width=700 height=400></iframe>'
ipywidgets.HTML(iframe)

## First, an HTML refresher
HTML is the basic language used to create a web page. 

It tells the web browser what text/media to display, where to display it, and how to display it (style)

HTML is very structured/hirarchical. 

Every page is made up of discrete "elements."

Elements are labeled with "tags."

For example:

    <p>You are beginning to learn HTML.</p>

A start tag also often contains "attributes" with info about the element.

Attributes usually have a name and value.

Example:

    <p class="my_red_sentences">You are beginning to learn HTML.</p>

We can make a table in HTML: we use the ```<tr>``` tag for table each table row, and the ```<td>``` for each column

```
<table id="mycats">
    <tbody>
        <tr>
            <th>name</th><th>color</th>
        </tr>
        <tr>
            <td>Button</td><td>white</td>
        </tr>
        <tr>
            <td>Peanut</td><td>Calico</td>
        </tr>
    </tbody>
</table>
```
<table id="mycats" width="50%">
<tbody>
<tr>
<th>name</th><th>color</th>
</tr>
<tr>
<td>Button</td><td>white</td>
</tr>
<tr>
<td>Peanut</td><td>Calico</td>
</tr>
</tbody>
</table>

A full HTML document has a structure more like this:

```
<html> 
  <head> </head>
  <body>
     <p class="red">You are beginning to learn HTML.</p>
     <h1> This is a header </h1>
     <a href="www.google.com"> Some link </a>
  </body>
</html>
```

Let's explore some live HTML!

Go to ```http://www.boxofficemojo.com/alltime/adjusted.htm``` in your browser,
right click and select Inspect Element. Point your cursor to the different elements on the page, what happens?
Also try right clicking and select view page source.

## Fetch a page with the GET request

When you open your browser, type an URL into the address bar, and hit Enter, the browser sends a "GET" request to the HTTP server. If the server responds "yeah, ok, I see you are requesting this page, let me send it to you", we get the data back (in HTML), and Viola! we see the content the of the page.

Doing this programatically in Python is super easy. There's a library for that: **Requests: HTTP for Humans**  
You can read more about the documentation [here](http://docs.python-requests.org/en/master/)


In [ ]:
# if needed: 
# !conda install requests -y
import requests

url = 'http://www.google.com/'
response = requests.get(url)
# various
print(response)
print(response.status_code)
print(response.url)
print(response.headers)
print(response.content)
print(response.text)
print(response.json())

In [ ]:
response.status_code

For information on HTTP status codes, see:  https://en.wikipedia.org/wiki/List_of_HTTP_status_codes

In [ ]:
print(response.text)

# Linkedin Industries
### Request Webpage and extract data from HTML Table
### Save to CSV

In [ ]:
import csv
import requests
import lxml.html
from bs4 import BeautifulSoup
from datetime import datetime

industry_url = r'https://developer.linkedin.com/docs/reference/industry-codes'

table_xpath = r'//*[@id="content"]/div[2]/div/section/div/div/div[2]/div[2]/table'

req = requests.get(industry_url)

lxml_html = lxml.html.fromstring(req.content)
root = lxml_html.getroottree()
table = root.xpath(table_xpath)
raw_table_html_bytes = lxml.html.tostring(table[0])
html_table = raw_table_html_bytes.decode('utf-8')

In [ ]:
now_time = datetime.utcnow().strftime('%Y%m%dT%H%M%S')

with open(r'data\\linkedin_industries.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    soup = BeautifulSoup(html_table, 'lxml')
    for i, row in enumerate(soup.findAll('tr')):
        table_row = [now_time]
        print(row)
        for cell in row.findAll('td'):
            table_row.append(f"{cell.get_text()}")
        writer.writerow(table_row + ['\n'])

# Reading data from an API

### Download data for set of Stock Tickers


In [3]:
# CSV
import os
import time
from datetime import datetime

import requests

tickers = "GOOG MSFT IBM TSLA".split()

today_date = datetime.today().strftime('%Y-%m-%d')

# download prices for each stock in portfolio
# as csv file
for ticker in tickers:
    print(f"Requesting Data for {ticker}")
    req = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&apikey=537201H9R203WT4C&datatype=csv")
    print(f"Saving {ticker} Data")
    
    # create filepath with date string
    output_filepath = os.path.join("data", f"{ticker}_PRICES_DAILY_{today_date}.csv")
    
    with open(output_filepath, 'w') as f:
        f.write(req.text)

    time.sleep(1) # sleep in-between requests

Requesting Data for GOOG
Saving GOOG Data
Requesting Data for MSFT
Saving MSFT Data
Requesting Data for IBM
Saving IBM Data
Requesting Data for TSLA
Saving TSLA Data


In [4]:
# JSON
import csv

# download prices for each stock in portfolio
# but this time json
for ticker in tickers:
    print(f"Requesting Data for {ticker}")
    req = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&apikey=537201H9R203WT4C&datatype=json")
    print(f"Saving {ticker} Data")

    # create filepath with date string
    output_filepath = os.path.join("data", f"{ticker}_PRICES_DAILY_{today_date}.json")
    with open(output_filepath, 'w') as f:
        f.write(req.text)

    time.sleep(1) # sleep in-between requests






'timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient\r\n2019-11-04,314.8000,321.9400,309.2600,317.4700,317.4700,8780129,0.0000,1.0000\r\n2019-11-01,316.3200,316.4800,309.8000,313.3100,313.3100,6380900,0.0000,1.0000\r\n2019-10-31,313.1000,319.0000,313.0000,314.9200,314.9200,5059900,0.0000,1.0000\r\n2019-10-30,313.0000,318.7900,309.9700,315.0100,315.0100,9637000,0.0000,1.0000\r\n2019-10-29,319.9900,324.3000,314.7500,316.2200,316.2200,12673300,0.0000,1.0000\r\n2019-10-28,327.5400,340.8400,322.6000,327.7100,327.7100,18870300,0.0000,1.0000\r\n2019-10-25,297.7200,330.0000,296.1100,328.1300,328.1300,29982400,0.0000,1.0000\r\n2019-10-24,298.3700,304.9300,289.2000,299.6800,299.6800,29720900,0.0000,1.0000\r\n2019-10-23,254.5000,256.1400,251.3500,254.6800,254.6800,5261100,0.0000,1.0000\r\n2019-10-22,254.3200,258.3300,250.8500,255.5800,255.5800,4600800,0.0000,1.0000\r\n2019-10-21,258.3300,259.5000,250.1800,253.5000,253.5000,5020300,0.0000,1.0000\r\n2019-10-18,260.